In [1]:
%matplotlib tk
import numpy as np
import pylab as pl
from scipy.integrate import odeint

The Lorenz attractor is a strange attractor that arises in a system of
equations describing the \mbox{2-dimensional} flow of fluid of uniform depth, with
an imposed vertical temperature difference.
In the early 1960s, Lorenz discovered the chaotic behavior of a
simplified 3-dimensional system of this problem,
now known as the Lorenz equations:

\begin{eqnarray}
  \frac{d}{d t}X &=& \sigma(Y-X) \label{eq:X}\\
  \frac{d}{d t}Y &=& -XZ + rX -Y \label{eq:Y}\\
  \frac{d}{d t}Z &=& XY - bZ \label{eq:Z}
\end{eqnarray}

\noindent 
The numerical solution of equations 
is calculated with
$\sigma = 3$,
$r = 26.5$ and
$b = 1$.

We generate the dataset integating the Lorenz system for a time T = 20 starting from 10 different initial conditions chosen randomly.
The 10 different initial solution will be appended to two matrices $Nt \times 3$ called $input$ and $output$ of the Neural Net.
The $input$ and the $output$ matrices are shifted by dt ($output(t) = input(t+dt)$) in order to teach the NN the dynamics of the system.

In [9]:
def lorenz(X, t, sigma, beta, rho):
    """The Lorenz equations."""
    u, v, w = X
    up = -sigma*(u - v)
    vp = rho*u - v - u*w
    wp = -beta*w + u*v
    return up, vp, wp

# Lorenz paramters and initial conditions
sigma, beta, rho = 10, 2.667, 28

Ni = 10 #number of initial conditions

inputs_  = []
outputs_ = []

for i in range(Ni):


    u0, v0, w0 =  10*(np.random.rand(3)-0.5) #0, 1, 1.05

    # Maximum time point and total number of time points
    tmax, n = 20, 2000

    # Integrate the Lorenz equations on the time grid t
    t = np.linspace(0, tmax, n)
    f = odeint(lorenz, (u0, v0, w0), t, args=(sigma, beta, rho))
    
    inputs_.append(f[0:-1,:]) # = np.vstack((inputs,f[0:-1,:]))
    outputs_.append(f[1:,:]) #= np.vstack((outputs,f[1:,:]))
    
    x, y, z = f.T

    fig = pl.figure(1)

    ax = fig.gca(projection='3d')

    # Make the line multi-coloured by plotting it in segments of length s which
    # change in colour across the whole time series.
    s = 100
    c = np.linspace(0,1,n)
    for i in range(0,n-s,s):
        ax.plot(x[i:i+s+1], y[i:i+s+1], z[i:i+s+1], color=(1,c[i],0), alpha=0.6)
     
    #ax.plot(x,y,z,"-",lw=1,alpha=0.5)
    ax.plot(u0,v0,w0,"ro",ms=5)
    #ax.set_axis_off() 

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

pl.savefig("Lorenz_dataset.png")

In [14]:
#a bit ofreshaping to save the results in a more convenient form 

a =  np.asarray(inputs_).shape[0]
b =  np.asarray(inputs_).shape[1]
c =  np.asarray(inputs_).shape[2]

In [17]:
#save the dataset to be imported into the NN

np.savetxt("inputs",np.asarray(inputs_).reshape(a*b,c))
np.savetxt("outputs",np.asarray(outputs_).reshape(a*b,c))